In [31]:
from collections import defaultdict, deque
from aocutils.grid import gridneigh
import sys

grid, neigh = gridneigh("in.txt", to_dict=True, parser=lambda x: [ch for ch in x])
grids = [grid]
poss = set()
wind = []
taken = set()
for node, val in grid.items():
    if val != "#":
        poss.add(node)
    if val in "<>^v":
        wind.append((node, val))
        taken.add(node)
takens = {}
takens[0] = taken
rows = max(r for r, c in grid)
cols = max(c for r, c in grid)
do = {">": (0, 1), "<": (0, -1), "^": (-1, 0), "v": (1, 0)}

for i in range(1, 2000):
    taken = set()
    newwind = []
    for node, val in wind:
        r, c = node
        if (new := (r + do[val][0], c + do[val][1])) in poss:
            newwind.append((new, val))
        else:
            if val == "v":
                newwind.append(((1, c), val))
            if val == "^":
                newwind.append(((rows - 1, c), val))
            if val == "<":
                newwind.append(((r, cols - 1), val))
            if val == ">":
                newwind.append(((r, 1), val))
        taken.add(newwind[-1][0])
        wind = newwind
    takens[i] = taken

In [32]:
# made one major mistake by always adding waiting to the queue. But that is not always allowed when the snowstorm passes over that position
def bfs(goal):
    seen = set()
    tocheck = deque([(0,1,0,1)])
    while tocheck:
        r,c, dis, leg = tocheck.popleft()
        for n in neigh[(r,c)]:
            newleg = leg
            if leg == 1 and n == (rows, cols-1):
                newleg = 2
            if leg == 2 and n == (0,1):
                newleg = 3
            if n in poss and (new := (n[0], n[1], dis + 1, newleg)) not in seen:
                if (n[0], n[1], newleg) == goal:
                    return dis + 1
                if n not in takens[dis+1]:
                    seen.add(new)
                    tocheck.append(new)
                else:
                    pass
        if (r,c, dis + 1,leg) not in seen and (r,c) not in takens[dis+1]:
            tocheck.append((r,c,dis+1, leg))    
            
print(bfs((rows, cols-1,2)), bfs((rows, cols-1,3)))

274 839


In [38]:
# A* makes it 4x faster, but also the priority is really unstable
import heapq
def bfs(goal):
    seen = set()
    tocheck = [(0,(0,1,0,1))]
    while tocheck:
        _, (r,c, dis, leg) = heapq.heappop(tocheck)
        for n in neigh[(r,c)]:
            newleg = leg
            if leg == 1 and n == (rows, cols-1):
                newleg = 2
            if leg == 2 and n == (0,1):
                newleg = 3
            if n in poss and (new := (n[0], n[1], dis + 1, newleg)) not in seen:
                if (n[0], n[1], newleg) == goal:
                    return dis + 1
                if n not in takens[dis+1]:
                    seen.add(new)
                    if newleg == 2:
                        prio = n[0] + n[1] -newleg*1000 + dis
                    else:
                        prio = -n[0] - n[1] -newleg*1000 + dis
                        
                    # print('prio', prio, new)
                    heapq.heappush(tocheck, (prio, new))
                else:
                    pass
        if (r,c, dis + 1,leg) not in seen and (r,c) not in takens[dis+1]:
            heapq.heappush(tocheck, (prio, (r,c,dis+1, leg)))
            
print(bfs((rows, cols-1,2)), bfs((rows, cols-1,3)))

274 839
